In [ ]:
import autogen
from autogen import UserProxyAgent, AssistantAgent

def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()


config_list_dalle3 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={"model": ["dall-e-3"]},
)

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={"model": ["gpt-4", "gpt-4-Vision", "gpt4", "gpt-4-32k"]},
)

# Assign the configuration to the respective config variables as per Step 1, paste the code below.
# <Step 1 code goes here>

# Initialize User Proxy Agent
# <Step 2 code goes here>



planner = AssistantAgent(
    name="Tour_Planner_Agent",
    is_termination_msg=termination_msg,
    # <Step 3 code goes here>
    llm_config=llm_config_gpt4,
    description="Tour Planner Agent to can plan travel in international travels.",
)

localtourist_Guide = AssistantAgent(
    name="Local_Tourist_Guide",
    is_termination_msg=termination_msg,
    # <Step 4 code goes here>
    llm_config=llm_config_gpt4,
    description="knowledgeable, personable, and insightful Local Tourist Guide Agent",
)

prompt_agent = AssistantAgent(
    name="Prompt_Agent",
    is_termination_msg=termination_msg,
    system_message='''You are a prompt engineer for image generation tasks using the DALL-E model.
      Your goal is to generate creative and accurate image prompts for the model based on user input.
 
      **Your Responsibilities:**
 
      1. **Analyze Itinerary:** Carefully read the itinerary and identify their desired image
      2. **Understand User Preferences:** Determine the user's preferred style, tone, and overall aesthetic (e.g., realistic, cartoon, abstract, whimsical).
      3. **Generate Image Prompts:** Craft one or more detailed image generation prompts based on the itinerary and preferences.
          - **Clarity is Key:** Make sure your prompts are clear, specific, and easy for the DALL-E model to interpret.
          - **Include Details:** Provide information about subject, setting, action, style, and composition.
          - **Use Descriptive Language:**  Choose words and phrases that evoke the desired visual style and imagery.
 
      Please make sure your response only contains prompt sentences, without including any description or introduction of this prompt.
      ''',
    llm_config=llm_config_gpt4,
    description="Generates prompts for DallE agent."
)
dalle_agent = ConversableAgent(
    name="Dalle_Agent",
    default_auto_reply= f"Image URL generated",
    description="Generates images using DallE model based on the input from prompt agent",
)
 
def generate_image(recipient, messages, sender, config):
    # Extract the llm_config from the config parameter
    llm_config = config.get("llm_config", {})
    config_list = llm_config.get("config_list", [])
    
    # Extract the relevant configuration details
    if config_list:
        model_config = config_list[0]
        api_version = model_config.get("api_version")
        api_key = model_config.get("api_key")
        azure_endpoint = model_config.get("base_url")
    else:
        raise ValueError("Configuration list is empty or missing required details.")
    
    # Initialize the AzureOpenAI client with the provided configuration
    client = AzureOpenAI(
        api_version=api_version,  
        api_key=api_key,  
        azure_endpoint=azure_endpoint,
    )
    print(messages[-1]['content'])
    result = client.images.generate(
        model="dall-e-3", # the name of your DALL-E 3 deployment
        prompt="""
    Strictly just convert the text to image and make it more visually appealing . It should look like reading the text. Dont not modify the text in any way.
        """+messages[-1]['content'],
        n=1
    )

    json_response = json.loads(result.model_dump_json())

    # Set the directory for the stored image
    image_dir = os.path.join(os.curdir, 'images')

    # If the directory doesn't exist, create it
    if not os.path.isdir(image_dir):
        os.mkdir(image_dir)

    # Initialize the image path (note the filetype should be png)
    image_path = os.path.join(image_dir, 'generated_image.png')

    # Retrieve the generated image
    image_url = json_response["data"][0]["url"]  # extract image URL from response
    generated_image = requests.get(image_url).content  # download the image
    with open(image_path, "wb") as image_file:
        image_file.write(generated_image)

    # Display the image in the default image viewer
    image = Image.open(image_path)
    image.show()
    return False, {"content": messages}

dalle_agent.register_reply(
        [autogen.Agent, None],
        reply_func=generate_image,
        config={"llm_config": llm_config_dalle3},
    )

def _reset_agents():
    boss.reset()
    planner.reset()
    localtourist_Guide.reset()
    prompt_agent.reset()
    dalle_agent.reset()


def chat_travel(preferences: dict):
    _reset_agents()
    preferences_message = (
        f"Create a two-day outing plan for Seattle in February. Include:  {preferences['activities']}\n- Meal preferences: {preferences['meals']}\n- Transport: {preferences['transport']}."
    )
    # <Step 5 code goes here>
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config_gpt4)

    # Start chatting with the boss as this is the user proxy agent.
    result= boss.initiate_chat(
        manager,
        message=preferences_message,
    )
    return result


# Example usage
preferences = {
    "activities": "outdoor attractions, museums, and scenic viewpoints",
    "meals": "vegetarian options with local specialties",
    "transport": "public transit and walking"
}

result=chat_travel(preferences)
